# Loading the Data

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
wec=pd.read_csv("C:\\Project DATA\\WEC Analysis 2021\\Data\\wec_hypercar_2021.csv")
spa=wec[wec['circuit']=='SPA FRANCORCHAMPS']
spa["pit_time"] = pd.to_timedelta(spa["pit_time"], errors="coerce")
spa["pit_time_sec"] = spa["pit_time"].dt.total_seconds()

# Analysis

## Driver Stint Consistency

In [3]:
fig = px.line(spa,
    x="lap_number",
    y="lap_time_s",
    title="Driver Stint Consistency",
    color="team_no",          
    line_group="driver_stint", 
    markers=True,
    labels={                  
        "lap_number": "Lap Number",
        "lap_time_s": "Lap Time (s)",
        "driver_stint": "Driver Stint",
        "team_no": "Team Number"
    }
)

fig.update_layout(
    xaxis_title="Lap Number",
    yaxis_title="Lap Time (s)",
    legend_title="Team Number"
)

fig.show()

In [4]:
fig = px.line(spa,
    x="lap_number",
    y="lap_time_s",
    title="Driver Stint Consistency",
    color="driver_stint",
    line_group="team_no",
    markers=True,
    labels={                  
        "lap_number": "Lap Number",
        "lap_time_s": "Lap Time (s)",
        "driver_stint": "Driver Stint",
        "team_no": "Team Number"
    }
)

fig.update_layout(
    xaxis_title="Lap Number",
    yaxis_title="Lap Time (s)",
    legend_title="Driver Name"
)

fig.show()

## Pit Stop Strategy Effectiveness

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

for team in spa["team_no"].unique():
    team_df = spa[spa["team_no"] == team]

    fig.add_trace(go.Scatter(
        x=team_df["lap_number"],
        y=team_df["lap_time_s"],
        mode="lines",
        line=dict(shape="hv"),
        name=f"Team {team}",
        legendgroup=str(team),  
        showlegend=True
    ))

    pit_laps = team_df[team_df["pit_time_sec"].notna() & (team_df["pit_time_sec"] > 0)]
    fig.add_trace(go.Scatter(
        x=pit_laps["lap_number"],
        y=pit_laps["lap_time_s"],
        mode="markers",
        marker=dict(size=10, symbol="x", color="red"),
        name="Pit Stop",
        legendgroup=str(team),  
        showlegend=False        
    ))

fig.update_layout(
    title="Pit Stop Strategy Effectiveness",
    xaxis_title="Lap Number",
    yaxis_title="Lap Time (s)",
    legend_title="Team Number"
)

fig.show()

## Stint Degradation

In [ ]:
stint_summary = (
    spa.groupby(["driver_stint", "team_no"]).agg(
        start_lap=("lap_number", "min"),
        end_lap=("lap_number", "max"),
        start_time=("lap_time_s", "first"),
        end_time=("lap_time_s", "last")
    )
    .reset_index()
)

stint_long = pd.concat(
    [stint_summary[["driver_stint", "team_no", "start_lap", "start_time"]].rename(columns={"start_lap": "lap", "start_time": "lap_time"}),

    stint_summary[["driver_stint", "team_no", "end_lap", "end_time"]].rename(columns={"end_lap": "lap", "end_time": "lap_time"})],
    ignore_index=True
)

fig = px.line(stint_long,
    x="lap",
    y="lap_time",
    color="driver_stint",   
    line_group="team_no", 
    markers=True,
    title="Stint Degradation: Start vs End",
    labels={                  
        "lap": "Lap Number",
        "lap_time": "Lap Time",
        "driver_stint": "Driver Stint",
        "team_no": "Team Number"
    }
)

fig.update_layout(
    xaxis_title="Lap Number",
    yaxis_title="Lap Time (s)"
)

fig.show()

In [ ]:
stint_summary = (
    spa.groupby("driver_stint")
    .agg(
        start_lap_time=("lap_time_s", "first"),
        end_lap_time=("lap_time_s", "last"),
        stint_length=("lap_time_s", "count"),
        team_no=("team_no", "first"), 
        driver=("driver_name", "first")
    )
    .reset_index()
)

stint_summary["delta_end_start"] = (stint_summary["end_lap_time"] - stint_summary["start_lap_time"])

fig = px.scatter(
    stint_summary,
    x="stint_length",
    y="delta_end_start",
    color="team_no",
    hover_data=["driver", "driver_stint"],
    title="Stint Degradation vs Stint Length",
    labels={                  
        "team_no": "Team Number",
        "stint_length": "Stint Length",
        "driver_stint": "Driver Stint",
        "team_no": "Team Number",
        "driver": "Driver",
        "delta_end_start": "Delta End Start"
    }
)

fig.update_layout(
    xaxis_title="Stint Length (laps)",
    yaxis_title="Lap Time Degradation (End − Start) [s]"
)

fig.show()

## Gap Evolution to Leader

In [ ]:
fig = px.line(spa,
    x="lap_number",
    y="class_gap",
    color="team_no",
    title="Gap Evolution to Race Leader",
    labels={
        "lap_number": "Lap Number",
        "class_gap": "Gap to Leader (s)",
        "team_no": "Team Number"
    }
)

fig.show()

## Drive Pace Delta within Team and Team Pace Delta

In [ ]:
spa["stint_lap_index"] = spa.groupby("driver_stint").cumcount()

first_lap_time = (spa[spa["stint_lap_index"] == 0].set_index("driver_stint")["lap_time_s"])

spa["lap_time_delta"] = (spa["lap_time_s"]- spa["driver_stint"].map(first_lap_time))

fig = px.box(spa,
    x="team_no",
    y="lap_time_delta",
    color="driver_name",
    title="Driver Pace Delta Within Team"
)

fig.update_layout(
    xaxis_title="Team",
    yaxis_title="Lap Time Δ vs Stint First Lap (s)"
)

fig.show()

In [ ]:
spa["stint_lap_index"] = spa.groupby("driver_stint").cumcount()

first_lap_time = (spa[spa["stint_lap_index"] == 0].set_index("driver_stint")["lap_time_s"])

spa["lap_time_delta"] = (spa["lap_time_s"] - spa["driver_stint"].map(first_lap_time))

fig = px.violin(spa,
    x="team_no",
    y="lap_time_delta",
    box=True,          
    points="outliers", 
    color="team_no",
    title="Team Pace Comparison (Lap Time Δ vs Stint First Lap)"
)

fig.update_layout(
    xaxis_title="Team",
    yaxis_title="Lap Time Δ (s)",
    showlegend=False
)

fig.show()